<a href="https://colab.research.google.com/github/GabrielBG0/Fakenews-Recognition/blob/main/Bart_%26_FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Deleting default sample data folder
!rm -rf /content/sample_data
!unzip FakeNewsNet.zip

Archive:  FakeNewsNet.zip
replace FakeNewsNet/dict_documents.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [42]:
# Install aditional librarys
!pip install transformers
!pip install datasets

In [43]:
import pickle
import numpy as np


documents = pickle.load(open("FakeNewsNet/dict_documents.pkl", "rb"))
indexes = np.load('FakeNewsNet/indexes.npy')
lbls = np.load('FakeNewsNet/labels.npy')

texts = []
labels = []
for i in range(len(indexes)):
    index = indexes[i]
    texts.append(documents[index])
    if lbls[i] == np.str_("-1"):
      labels.append(0)
    else :
      labels.append(1)

print("lable 1: " + str(labels[0]))
print("text 1: " + texts[0])
print(type(labels[0]))

lable 1: 0
text 1: When I revealed in this column last year that Dame Helen Mirren was the new face of L’Oreal, I had hundreds of letters from readers, ecstatic that beauty giants are celebrating older women.  As these exclusive pictures show, from next week (February 5) Helen will be appearing in her first television advertising campaign for the company.  The 69-year-old actress will be seen championing the role of the older woman — and the candid, unretouched shots from the advert show that wrinkles, ageing hands and natural make-up can be beautiful.  Scroll down for video  Dame Helen Mirren photographed for the L'Oreal Paris age perfect campaign. In this shot Helen wears the Age Perfect day cream, the candid, unretouched shots are a refreshing change and champion the beauty of the older woman  I suspect this approach will sell more products than the glossier, airbrushed campaigns that many women feel don’t relate to them and their real lives.  During the filming of the advert, Dame 

In [44]:
for text in texts:
  if len(text.split(' ')) > 512:
    print("there are too long texts! truncate then")
    break

there are too long texts! truncate then


In [45]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [46]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [47]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [48]:
import torch

class FNNDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FNNDataset(train_encodings, train_labels)
val_dataset = FNNDataset(val_encodings, val_labels)
test_dataset = FNNDataset(test_encodings, test_labels)


In [49]:
# Checks if there is a gpu available
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')

Found GPU at: /device:GPU:0


In [50]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored